In [1]:
from valor import connect
connect("http://localhost:8000")

	== Running with a mismatched client != API version may have unexpected results.
	== Please update your client to [1;0.27.2.dev37+g6c9eaddf to avoid aberrant behavior.



Successfully connected to host at http://localhost:8000/


In [2]:
from valor import (
    Annotation,
    Client,
    Dataset,
    Datum,
    Filter,
    GroundTruth,
    Label,
)
from valor.schemas import And, Box

client = Client()

# Example - Swimmers and boats.

This example demonstrates how to create and query a dataset containing images annotated with boats, swimmers, and fish.

In [3]:
# Define the data for the example
contains_boat_swimmer = [
    ("uid1", False, False),
    ("uid2", True, False),
    ("uid3", False, True),
    ("uid4", True, True),
]

# Create a bounding box for annotations
box = Box.from_extrema(0, 10, 0, 10)

# Define labels for annotations
swimmer_label = Label(key="class", value="swimmer")
boat_label = Label(key="class", value="boat")
fish_label = Label(key="class", value="fish")

# Create a dataset for the images
dataset1 = Dataset.create("dataset1")

# Add annotations to the dataset
for uid, is_boat, is_swimmer in contains_boat_swimmer:
    annotations = [
        Annotation(
            labels=[boat_label if is_boat else fish_label],
            bounding_box=box,
            is_instance=True,
        ),
        Annotation(
            labels=[swimmer_label if is_swimmer else fish_label],
            bounding_box=box,
            is_instance=True,
        ),
    ]
    dataset1.add_groundtruth(GroundTruth(datum=Datum(uid=uid), annotations=annotations))

Show all datums in the dataset.

In [4]:
dataset1.get_datums()

[Datum({'uid': 'uid4', 'metadata': {}}),
 Datum({'uid': 'uid3', 'metadata': {}}),
 Datum({'uid': 'uid2', 'metadata': {}}),
 Datum({'uid': 'uid1', 'metadata': {}})]

Query the dataset for images containing just fish

In [5]:
just_fish = dataset1.get_datums(
    Filter(
        datums=And(
            Label.key == "class",
            Label.value != "boat",
            Label.value != "swimmer",
        )
    )
)
assert len(just_fish) == 1
assert just_fish[0].uid == "uid1"

Query the dataset for images containing no swimmers (only boats)

In [6]:
no_swimmers = dataset1.get_datums(
    Filter(
        datums=And(
            Label.key == "class",
            Label.value == "boat",
            Label.value != "swimmer",
        )
    )
)
assert len(no_swimmers) == 1
assert no_swimmers[0].uid == "uid2"

Query the dataset for images containing no boats (only swimmers)

In [7]:
no_boats = dataset1.get_datums(
    Filter(
        datums=And(
            Label.key == "class",
            Label.value != "boat",
            Label.value == "swimmer",
        )
    )
)
assert len(no_boats) == 1
assert no_boats[0].uid == "uid3"

Query the dataset for images containing both swimmers and boats

In [8]:
swimmers_and_boats = dataset1.get_datums(
    Filter(
        datums=And(
            Label.key == "class",
            Label.value == "boat",
            Label.value == "swimmer",
        )
    )
)
assert len(swimmers_and_boats) == 1
assert swimmers_and_boats[0].uid == "uid4"

# Example - Swimmers w/ Boats of different sizes.

This example demonstrates how to create and query a dataset containing images annotated with boats, swimmers, and fish.

In this example, the boats are bounded by either a small bbox (5x5) or large bbox (10x10).

In [9]:
contains_boat_swimmer = (
    ("uid1", False, False),
    ("uid2", True, False),
    ("uid3", False, True),
    ("uid4", True, True),
)

small_box = Box.from_extrema(0, 5, 0, 5)
large_box = Box.from_extrema(0, 10, 0, 10)

swimmer = Label(key="class", value="swimmer")
boat = Label(key="class", value="boat")
fish = Label(key="class", value="fish")

dataset2 = Dataset.create("dataset2")
for uid, is_large_boat, is_swimmer in contains_boat_swimmer:
    dataset2.add_groundtruth(
        GroundTruth(
            datum=Datum(uid=uid),
            annotations=[
                Annotation(
                    labels=[boat],
                    bounding_box=large_box if is_large_boat else small_box,
                    is_instance=True,
                ),
                Annotation(
                    labels=[swimmer if is_swimmer else fish],
                    bounding_box=small_box,
                    is_instance=True,
                ),
            ],
        )
    )

No swimmer, small boats

In [10]:
no_swimmer_small_boats = dataset2.get_datums(
    Filter(
        datums=And(
            Label.key == "class",
            Label.value != "swimmer",
        ),
        annotations=And(
            Label.key == "class",
            Label.value == "boat",
            Annotation.bounding_box.area < 50,
        ),
    )
)
assert len(no_swimmer_small_boats) == 1
assert no_swimmer_small_boats[0].uid == "uid1"

No swimmer, large boats

In [11]:
no_swimmer_large_boats = dataset2.get_datums(
    Filter(
        datums=And(
            Label.key == "class",
            Label.value != "swimmer",
        ),
        annotations=And(
            Label.key == "class",
            Label.value == "boat",
            Annotation.bounding_box.area > 50,
        ),
    )
)
assert len(no_swimmer_large_boats) == 1
assert no_swimmer_large_boats[0].uid == "uid2"

Swimmer with small boats

In [12]:
swimmer_with_small_boats = dataset2.get_datums(
    Filter(
        datums=And(
            Label.key == "class",
            Label.value == "swimmer",
        ),
        annotations=And(
            Label.key == "class",
            Label.value == "boat",
            Annotation.bounding_box.area < 50,
        ),
    )
)
assert len(swimmer_with_small_boats) == 1
assert swimmer_with_small_boats[0].uid == "uid3"

Swimmer with large boat

In [13]:
swimmers_and_boats = dataset2.get_datums(
    Filter(
        datums=And(
            Label.key == "class",
            Label.value == "swimmer",
        ),
        annotations=And(
            Label.key == "class",
            Label.value == "boat",
            Annotation.bounding_box.area > 50,
        ),
    )
)
assert len(swimmers_and_boats) == 1
assert swimmers_and_boats[0].uid == "uid4"